In [286]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import random

df = pd.read_csv("competitions/ml-2018spring-hw1/train.csv", encoding="big5")

for row in np.arange(10, len(df), FEATURE_PER_DAY):
    df.iloc[row, 3:] = pd.to_numeric(df.iloc[row, 3:], errors="coerce")

df.fillna(0, inplace=True)

In [287]:
FEATURE_PER_DAY = 18
WINDOW_SIZE = 9
FEATURE_NUM = 18 * 9

In [288]:
x_data = []
y_data = []

for i in np.arange(0, len(df), FEATURE_PER_DAY):
    arr = np.array(df.iloc[i:i+FEATURE_PER_DAY, 3:3+WINDOW_SIZE], dtype=float)
    arr = np.insert(arr, 0, 1) # x0 for bias
    x_data.append(arr)
    y_data.append(np.array(df.iloc[i+9, 3+WINDOW_SIZE+1], dtype=float))

In [513]:
# Step 1: Model
# Loss function L(f) = sum of (y - f)^2
theta = np.array([0.0] * (FEATURE_NUM + 1)) # all parameters init to 1
# for i in range(len(theta)):
#     theta[i] = random.randint(0, 99)
lr = 0.0001
iteration = 1000

lr_ada = np.zeros(FEATURE_NUM + 1)

In [518]:
theta = np.load('theta.npy')

In [519]:
print(theta)

for i in range(iteration):
    grad = np.zeros(FEATURE_NUM + 1)
    
    for n in range(len(x_data)):
        for f in range(len(grad)):
            grad[f] = grad[f] - 2.0 * (y_data[n] - np.dot(theta, x_data[n])) * x_data[n][f]
    
    for f in range(len(grad)):
        lr_ada[f] = lr_ada[f] + grad[f] ** 2

    for index in range(len(theta)):
        theta[index] = theta[index] - lr / np.sqrt(lr_ada[index]) * grad[index]
    
    print('*', end='')
    
print(theta)

[-1.11153342e-01  3.98534954e-03  6.59485255e-02  1.03951345e-01
 -6.17658158e-02 -7.68843159e-03 -2.89879665e-03  3.81706554e-02
 -5.23439114e-02 -3.45930704e-02 -3.19378719e-02  3.92811834e-03
  1.12323022e-01 -1.08645259e-02  1.21694299e-01  3.96309011e-02
  1.17888884e-01  1.71348181e-01  3.94279133e-01 -6.72265804e-01
 -4.64302114e-01 -2.77490230e-01 -7.14826613e-01 -7.58501493e-01
 -8.99605871e-02 -2.32478978e-01  1.91796605e-01 -1.22610693e-01
 -8.75623038e-01 -4.48019694e-01  3.06369188e-01 -1.03950098e+00
 -4.63288657e-01  8.26405740e-01  4.67494028e-01  1.01325750e+00
  2.68484086e-01  1.26416639e-01  1.27323615e-01  4.77685154e-02
 -4.69918350e-02  2.04276437e-01  5.13895296e-02 -1.07036612e-01
 -1.08839955e-01  8.72024504e-02 -1.02958612e-01 -5.67395125e-02
  6.61515626e-02  7.21237289e-02  1.86311628e-01  8.91313502e-03
 -1.69184873e-01  1.06918483e-03  2.71770498e-02 -1.32702939e-01
 -1.16733583e-01  6.35647751e-02 -5.00271270e-02  2.02648520e-01
  8.84493862e-02 -1.72533

In [504]:
def computeCost(x_data, y_data, theta):
    cost = 0.0
    for i in range(len(x_data)):
        cost += (np.dot(theta, x_data[i]) - y_data[i]) ** 2
    return cost

16895.726658446325

In [516]:
record = [16895.726658446325]

In [520]:
record.append(computeCost(x_data, y_data, theta))
print(record)

[16895.726658446325, 16860.948723632286, 16873.55664384602]


In [505]:
for i in range(len(x_data)):
    print(np.dot(theta, x_data[i]), y_data[i])

61.456864691536765 67.0
31.76770536535094 37.0
34.234389608858876 19.0
31.966307748713287 43.0
66.6241790938204 79.0
74.94908103502429 83.0
42.52371487445913 55.0
63.59642443263599 68.0
19.44541924181975 23.0
7.579326208641965 13.0
41.61371139613113 43.0
25.263737793808307 41.0
23.706283711783136 21.0
17.548060459985596 0.0
10.350507230845686 16.0
40.58638482544358 31.0
43.21926329762412 44.0
22.74405804474112 21.0
79.40376642812006 78.0
54.469195370086155 51.0
48.31662267356162 49.0
25.298912390821005 24.0
25.84948350472835 36.0
17.963986075978724 7.0
14.00851743552091 15.0
8.537160264280255 14.0
25.28777770850461 28.0
17.89686286587534 12.0
10.917792331047902 5.0
-6.148781033835647 3.0
11.615298889420828 21.0
16.26941164791965 12.0
0.20548955321291706 7.0
19.366794733621123 20.0
10.418020585621377 14.0
27.33586405126867 41.0
46.14941247434336 49.0
42.23860592644958 41.0
8.987783759654716 6.0
22.780507038152354 23.0
42.490435553833166 42.0
12.914271717739531 12.0
14.18587923814083 -3.

In [506]:
np.save('theta', theta)

In [507]:
dt = pd.read_csv("competitions/ml-2018spring-hw1/test.csv", header=None, encoding="big5")

for row in np.arange(10, len(dt), FEATURE_PER_DAY):
    dt.iloc[row, 2:] = pd.to_numeric(dt.iloc[row, 2:], errors="coerce")
    
dt.fillna(0, inplace=True)

In [510]:
x_test = []
y_test = []

for i in np.arange(0, len(dt), FEATURE_PER_DAY):
    arr = np.array(dt.iloc[i:i+FEATURE_PER_DAY, 2:2+WINDOW_SIZE], dtype=float)
    arr = np.insert(arr, 0, 1) # x0 for bias
    x_test.append(arr)
    
for i in range(len(x_test)):
    y_test.append(np.dot(theta, x_test[i]))

In [511]:
for i in range(len(y_test)):
    if (y_test[i] < 0):
        y_test[i] = 0

In [512]:
arr = [["id_" + str(i), y_test[i]] for i in range(len(y_test))]
dw = pd.DataFrame(arr, columns = ["id", "value"])
dw.to_csv("output.csv", index=False)